In [3]:
# Cell 2: Imports
import os
import json
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType
import torch


In [4]:
# Cell 3: Preprocessing UFAL dataset
# Set your data paths
eng_file = "ufal_dataset/ufal-nla-v1.eng"
msa_file = "ufal_dataset/ufal-nla-v1.arb"
da_file  = "ufal_dataset/ufal-nla-v1.apc"

# Read files
with open(eng_file, encoding="utf-8") as f: eng = [line.strip() for line in f if line.strip()]
with open(msa_file, encoding="utf-8") as f: msa = [line.strip() for line in f if line.strip()]
with open(da_file, encoding="utf-8") as f: da  = [line.strip() for line in f if line.strip()]

# Check alignment
assert len(eng) == len(msa) == len(da), "Mismatch in number of lines"

# Prepare dataset in JSONL format for multilingual translation
dataset_list = []
for e, m, d in zip(eng, msa, da):
    dataset_list.append({
        "text": f"Instruction: Translate the following sentence to Syrian Arabic.\nInput: {m}\nOutput:"
    })
    dataset_list.append({
        "text": f"Instruction: Translate the following sentence to MSA.\nInput: {d}\nOutput:"
    })
    dataset_list.append({
        "text": f"Instruction: Translate the following sentence to English.\nInput: {d}\nOutput:"
    })

# Convert to HuggingFace dataset
dataset = Dataset.from_list(dataset_list)
dataset = dataset.train_test_split(test_size=0.1)  # 90/10 split

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 325620
    })
    test: Dataset({
        features: ['text'],
        num_rows: 36180
    })
})


In [5]:
# Cell 4: Load tokenizer
model_name = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/325620 [00:00<?, ? examples/s]

Map:   0%|          | 0/36180 [00:00<?, ? examples/s]

In [6]:
# Cell 5: Prepare LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj","v_proj"]
)


In [7]:
# Cell 6: Load model with LoRA
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # uses less GPU memory
    device_map="auto"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,587,520 || all params: 3,217,337,344 || trainable%: 0.1426


In [ ]:
# Cell 7: Training setup
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./llama3-syrian-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=5,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    save_total_limit=2,
    report_to="none",
    max_steps=10000,
    save_safetensors=False  # << disable safetensors
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)


In [10]:
# Cell 8: Start fine-tuning
trainer.train()
trainer.save_model("./llama3-syrian-finetuned")


In [12]:
# Cell 9: Load fine-tuned model for inference
from transformers import pipeline

model = AutoModelForCausalLM.from_pretrained("./llama3-syrian-finetuned", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=50)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [16]:
# ENG -> DA
prompt = "Instruction: Translate the following sentence to Syrian Arabic.\nInput:What's that idiot doing??\nOutput:"
output = generator(prompt)
print("ENG -> DA:", output[0]["generated_text"].split("Output:")[-1].strip())

# DA -> MSA
prompt = "Instruction: Translate the following sentence to MSA.\nInput: بدك تضل بالغابة؟\nOutput:"
output = generator(prompt)
print("DA -> MSA:", output[0]["generated_text"].split("Output:")[-1].strip())

# MSA -> ENG
prompt = "Instruction: Translate the following sentence to English.\nInput:شو رح نعمل بعدين\nOutput:"
output = generator(prompt)
print("MSA -> ENG:", output[0]["generated_text"].split("Output:")[-1].strip())


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


ENG -> DA: ماذا يعمل هذا المثقرف؟?? 

Note: Please keep in mind that translating to Syrian Arabic may not be the same as translating to Modern Standard Arabic, and may be more informal or dialectical.
DA -> MSA: بالغابة؟ يبدو لنا. تختلف الأشعة. بالغابة. يبدو لنا. تختلف الأشعة. بالغابة؟ يبدو لنا. تختلف الأشعة
MSA -> ENG: What will you do next?


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
lora_path = "./llama3-syrian-finetuned"
merged_output_path = "./uflh_finetunedmodel"

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, lora_path)

# Merge LoRA weights into base model
model = model.merge_and_unload()

# Save merged model
model.save_pretrained(merged_output_path, safe_serialization=True)

# Save tokenizer
tokenizer = AutoTokenizer.from_pretrained(lora_path)
tokenizer.save_pretrained(merged_output_path)


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('./uflh_finetunedmodel/tokenizer_config.json',
 './uflh_finetunedmodel/special_tokens_map.json',
 './uflh_finetunedmodel/chat_template.jinja',
 './uflh_finetunedmodel/tokenizer.json')